# Picking Algorithms

In this exercise you get code snippets for two optimizations that fail silently. You need to fix them by selecting a different optimization algorithm. To make it a bit easier, we provide the criterion value at the optimum.

If you find one that works, continue to search for one that works better. 

## Resources

- [List of algorithms](https://estimagic.readthedocs.io/en/stable/algorithms.html)
- [Documentation of algo_options](https://estimagic.readthedocs.io/en/stable/how_to_guides/optimization/how_to_specify_algorithm_and_algo_options.html)

In [1]:
import numpy as np
import estimagic as em

## Problem 1

This is a modified version of the `powell_singular` function from the More and Wild benchmark set. 

In [2]:
def powell_steps(x):
    x = x.round(4)
    fvec = np.zeros(4)
    fvec[0] = x[0] + 10 * x[1]
    fvec[1] = np.sqrt(5.0) * (x[2] - x[3])
    fvec[2] = (x[1] - 2 * x[2]) ** 2
    fvec[3] = np.sqrt(10.0) * (x[0] - x[3]) ** 2
    out = {"root_contributions": fvec, "value": np.dot(fvec, fvec)}
    return out

powell_start = np.array([3, -1, 0, 1])

In [3]:
# optimal criterion value
powell_criterion = 0

In [4]:
res_powell = em.minimize(
    criterion=powell_steps,
    params=powell_start,
    algorithm="scipy_lbfgsb",
)

res_powell

Minimize with 4 free parameters terminated successfully after 1 criterion evaluations, 1 derivative evaluations and 0 iterations.

The value of criterion improved from 215.00000000000003 to 215.00000000000003.

The scipy_lbfgsb algorithm reported: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL

---
The optimization thinks it terminated "successfully" but when you look at `res.params` you see that it actually got stuck at the start values.

In [5]:
res_powell.params

array([ 3., -1.,  0.,  1.])

## Task 1:

- Try to understand why the algorithm `scipy_lbfgsb` got stuck at the start parameters
- Find an algorithm that converges to the optimum

## Solution 1:

- The algorithm uses gradient information, however, because `powell_steps` rounds the input, the gradient is zero. Hence, it thinks it is at an optimum.
- Use a gradient-free optimizer that utilizes the least-squares structure (`nag_dfols`)

In [6]:
res_powell = em.minimize(
    criterion=powell_steps,
    params=powell_start,
    algorithm="nag_dfols",
)

In [7]:
res_powell.params

array([ 1.02418627e-03, -7.09648552e-05,  8.86890203e-04,  9.33756248e-04])

In [8]:
res_powell.criterion

1.3033100000000002e-11

## Problem 2

This is the `bratu_2d` problem from the Cartis and Roberts benchmark set. 

In [9]:
def bratu(x):
    alpha = 4.
    x = x.reshape((int(np.sqrt(len(x))), int(np.sqrt(len(x)))))
    p = x.shape[0] + 2
    h = 1 / (p - 1)
    c = alpha * h**2
    xvec = np.zeros((x.shape[0] + 2, x.shape[1] + 2))
    xvec[1 : x.shape[0] + 1, 1 : x.shape[1] + 1] = x
    fvec = np.zeros_like(x)
    for i in range(2, p):
        for j in range(2, p):
            fvec[i - 2, j - 2] = (
                4 * xvec[i - 1, j - 1]
                - xvec[i, j - 1]
                - xvec[i - 2, j - 1]
                - xvec[i - 1, j]
                - xvec[i - 1, j - 2]
                - c * np.exp(xvec[i - 1, j - 1])
            )
    fvec = fvec.flatten()
    out = {"root_contributions": fvec, "value": np.dot(fvec, fvec)}
    return out

bratu_start = np.zeros(64)

In [10]:
# optimal criterion value 
bratu_criterion = 0

In [11]:
res_bratu = em.minimize(
    criterion=bratu,
    params=bratu_start,
    algorithm="scipy_neldermead",
    algo_options={"stopping.max_criterion_evaluations": 5000},
)

res_bratu

Minimize with 64 free parameters terminated unsuccessfully after 5000 criterion evaluations and 4697 iterations.

The value of criterion improved from 0.1560737692424935 to 0.14373729795343515.

The scipy_neldermead algorithm reported: Maximum number of function evaluations has been exceeded.

Independent of the convergence criteria used by scipy_neldermead, the strength of convergence can be assessed by the following criteria:

                             one_step     five_steps 
relative_criterion_change  0.0001292     0.0007854   
relative_params_change       0.02281        0.0461   
absolute_criterion_change  1.857e-05     0.0001129   
absolute_params_change      0.002281       0.00461   

(***: change <= 1e-10, **: change <= 1e-8, *: change <= 1e-5. Change refers to a change between accepted steps. The first column only considers the last step. The second column considers the last five steps.)

---
The optimization will terminate because it reaches the maximum number of criterion evaluations. Of course, you could increase that number even further, but you can do better! Also: One million evaluations would not even be enough!

## Task 2:

- Try to understand why `scipy_neldermead` does not converge on this problem
- Find an algorithm that converges to the optimum

## Solution 2:

- The algorithm `scipy_neldermead` does not use gradient information.
- Use an algorithm that utilizes the gradient information (`scipy_lbfgsb`)

In [12]:
res_bratu = em.minimize(
    criterion=bratu,
    params=bratu_start,
    algorithm="scipy_lbfgsb",
)

res_bratu

Minimize with 64 free parameters terminated successfully after 41 criterion evaluations, 41 derivative evaluations and 37 iterations.

The value of criterion improved from 0.1560737692424935 to 4.3730465204731034e-10.

The scipy_lbfgsb algorithm reported: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH

Independent of the convergence criteria used by scipy_lbfgsb, the strength of convergence can be assessed by the following criteria:

                             one_step     five_steps 
relative_criterion_change  1.849e-08*    7.418e-07*  
relative_params_change     0.0001616      0.005466   
absolute_criterion_change  1.849e-09**   7.418e-08*  
absolute_params_change     4.195e-05      0.001454   

(***: change <= 1e-10, **: change <= 1e-8, *: change <= 1e-5. Change refers to a change between accepted steps. The first column only considers the last step. The second column considers the last five steps.)